[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Fuenfgeld/2022TeamBDataEngineeringBC/blob/main/Uebungen/Loesungen/Loesungen%20zu%20Teil%202.ipynb)

# Lösungen zu Teil 2

## Bitte führen Sie die nächsten beiden Codeabschnitte aus

In [ ]:
#Load Libraries
import sqlite3
import pandas as pd

# Load GitHub repo
%cd /content/
%rm -r ./2022TeamBDataEngineeringBC
!git clone https://github.com/Fuenfgeld/2022TeamBDataEngineeringBC.git
%cd ./2022TeamBDataEngineeringBC/

In [ ]:
# Connect to Database
conn = sqlite3.connect('sqlite_workshop.db')
cursor = conn.cursor()
print("Opened database successfully");

# Delete table if exists 
cursor.execute("DROP TABLE IF EXISTS food")
cursor.execute("DROP TABLE IF EXISTS orders")
cursor.execute("DROP TABLE IF EXISTS animals")
cursor.execute("DROP TABLE IF EXISTS customer")
cursor.execute("DROP TABLE IF EXISTS customer_buys_food")
cursor.execute("DROP TABLE IF EXISTS employee")
cursor.execute("DROP TABLE IF EXISTS sales")
cursor.execute("DROP TABLE IF EXISTS food_new")
cursor.execute("DROP TABLE IF EXISTS customer_new")
cursor.execute("DROP TABLE IF EXISTS employee_new")
cursor.execute("DROP TABLE IF EXISTS Liga")
cursor.execute("DROP TABLE IF EXISTS Verein")
cursor.execute("DROP TABLE IF EXISTS Spieler")

# Daten einlesen
food = pd.read_csv('tables/initial_tables/Futter.csv', sep=";")
customer = pd.read_csv('tables/initial_tables/Kunden.csv', sep=";")
animals = pd.read_csv('tables/initial_tables/Tiere.csv', sep=";")
orders = pd.read_csv('tables/initial_tables/Order.csv', sep=";")
employee = pd.read_csv('tables/initial_tables/Verkaeufer.csv', sep=";")
customer_buys_food = pd.read_csv('tables/initial_tables/Customer_buys_Food.csv', sep=";")


# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS customer (
			customer_ID int primary key,
            name nvarchar(50),
			postal_code nvarchar(5),
			city nvarchar(50)
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS employee (
			employee_ID int primary key,
            name nvarchar(50),
            gender nvarchar(1),
			wage int
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS food (
			food_ID int primary key,
            name nvarchar(50),
			price_per_kg float,
            producer nvarchar(50),
            origin nvarchar(50)
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS animals (
            name nvarchar(50) primary key,
			food int,
			foreign key(food) references food(food_ID)
			)
               ''') 
cursor.execute('''
		CREATE TABLE IF NOT EXISTS orders (
            order_Nr int primary key,
            date Date,
            transact int
			)
               ''')


cursor.execute('''
		CREATE TABLE IF NOT EXISTS customer_buys_food (
            C_ID int,
            F_ID int,
			amount int,
            O_ID int,
            E_ID int,
            foreign key(C_ID) references customer(customer_ID)
			foreign key(F_ID) references food(food_ID)
            foreign key(O_ID) references orders(order_Nr)
            foreign key(E_ID) references employee(employee_ID)
			)
               ''') 

# Insert Dataframe in Table
customer.to_sql('customer', conn, if_exists='append', index=False)
animals.to_sql('animals', conn, if_exists='append', index=False)
food.to_sql('food', conn, if_exists='append', index=False)
orders.to_sql('orders', conn, if_exists='append', index=False)
employee.to_sql('employee', conn, if_exists='append', index=False)
customer_buys_food.to_sql('customer_buys_food', conn, if_exists='append', index=False)

### Hiermit werden die Tabelleninhalte angezeigt

In [ ]:
print("Tabelle customer")
print(pd.read_sql_query("SELECT * FROM customer", conn))
print("--------------------------------------------------------")
print("Tabelle food")
print(pd.read_sql_query("SELECT * FROM food", conn))
print("--------------------------------------------------------")
print("Tabelle animals")
print(pd.read_sql_query("SELECT * FROM animals", conn))
print("--------------------------------------------------------")
print("Tabelle orders")
print(pd.read_sql_query("SELECT * FROM orders", conn))
print("--------------------------------------------------------")
print("Tabelle employee")
print(pd.read_sql_query("SELECT * FROM employee", conn))
print("--------------------------------------------------------")
print("Tabellen customer_buys_food")
print(pd.read_sql_query("SELECT * FROM customer_buys_food", conn))
print("--------------------------------------------------------")

# Aufgabe zu Teil 2

## ER-Diagramm

[<img src="https://raw.githubusercontent.com/Fuenfgeld/2022TeamBDataEngineeringBC/main/Presentation/Bilder/nur_tabellen.draw_io.drawio_3.png">](https://raw.githubusercontent.com/Fuenfgeld/2022TeamBDataEngineeringBC/main/Presentation/Bilder/nur_tabellen.draw_io.drawio_3.png)

## SUBQUERIES

#### Beispiel: Welche Kunden (C_ID) haben Futter gekauft, das in China hergestellt wurde?

In [ ]:
print(pd.read_sql_query('''
    SELECT C_ID 
    FROM customer_buys_food
    WHERE F_ID =
       (SELECT food_ID FROM food WHERE origin == 'China')
    ''', conn))

### Aufgabe 2.1: Welche Bestellungen (O_ID) wurden von den Verkäufern abgewickelt, die mehr als 2000 verdienen?

In [ ]:
print(pd.read_sql_query('''
     SELECT O_ID 
     FROM customer_buys_food
     WHERE E_ID IN
       (SELECT employee_ID FROM employee WHERE employee.wage > 2000)
     ''', conn))

### Aufgabe 2.2: Welche Kunden (name) haben Futter von weiblichen Verkäufern verkauft bekommen?

In [ ]:
print(pd.read_sql_query('''
    SELECT name
    FROM customer
    WHERE customer_ID IN (
        SELECT C_ID
        FROM customer_buys_food
        WHERE E_ID IN (
            SELECT employee_ID 
            FROM employee 
            WHERE gender == 'w')
        )
    ''', conn))